In [7]:
from __future__ import print_function
import librosa
import numpy as np
import cv2 as cv
import shutil
import glob
from sys import exit
import subprocess
from tkinter import filedialog
from tkinter import *
from os import listdir
from os.path import isfile, join
import math
import madmom
from madmom.models import BEATS_LSTM
import os
try: 
    import requests
except ImportError:
    print('Requests must be installed. PLease run: pip install requests')
    exit()
#"im in awe at the size of this lad"
#"ABSOLUTE UNIT"

y=''
sr=''#older versions but im not cleaning up so i dont break anything it 3 am and i dont want to deal with it
dir="reddit_images/"
fps = 30 #potato mode
vid_height=1080
vid_width = 1920
threshold=.2 #beat threshhold


fourcc = cv.VideoWriter_fourcc(*'XVID')#open cv video garbage
out = cv.VideoWriter('output.avi', fourcc, 20.0, (vid_width,  vid_height))
def beats(song):#mine but some from offical tutorial
    global threshold
    global fps
    proc = madmom.features.beats.RNNBeatProcessor(online=True, nn_files=[BEATS_LSTM[0]],fps=fps)
    #print(proc)  
    songdata=proc(song)
    beats=madmom.features.beats.detect_beats(songdata, fps, look_aside=0.2)#np.where(songdata>=threshold)
    #print(beats[0])
    #print(len(beats[0]))
    return(beats)#this is in FRAMES not seconds
#why does minecraft music slap at 3am



    

def makeUrl(afterID, subreddit):#not mine
    newUrl = subreddit.split('/.json')[0] + "/.json?after={}".format(afterID)
    return newUrl
    

def splitUrl(imageUrl):#not mine
    if 'jpg' or 'webm' or 'mp4' or 'gif' or 'gifv' or 'png' in imageUrl:
        tempfix=imageUrl.split('/')[-1]  # Returns filename
        return tempfix.split('?')[0]


def downloadImage(imageUrl, imageAmount):#not mine   # Function to download the images from
    global dir
    print(imageUrl)                          # each url passed to it
    filename = splitUrl(imageUrl)
    #print(filename)
    if filename:
        if ('jpg' or 'webm' or 'mp4' or 'gif' or 'gifv' or 'png') in filename:
            r = requests.get(imageUrl, stream=True)  # Downloads image  #nice
            with open(dir+filename, 'wb') as f:
                shutil.copyfileobj(r.raw, f)
                #print("Successfully downloaded: " + imageUrl)
                imageAmount += 1
    return imageAmount


def reddit_rip(subname,limit,sorting="hot",time="default"):#some of my code
    subreddit = "https://www.reddit.com/r/"+subname+"/"+sorting
    if time != "default":
        subreddit+="/?t="+time
    print(subreddit)
    print(limit)
    limit=int(limit)
    #amount = int(input("How many images would you like to download: "))
    subJson = ''
    x = 0
    while x < limit:
        if subJson:
            url = makeUrl(subJson['data']['after'], subreddit)
        else:
            url = makeUrl('', subreddit)
        subJson = requests.get(url, headers={'User-Agent': 'MyRedditScraper'}).json()
        #print(subJson)
        post = subJson['data']['children']
        postCount = range(len(post))
        
        # Sets postCount equal to the number of posts

        for i in postCount:  # Iterates over the number of posts received the sub
            imageUrl = (post[i]['data']['url'])
            _imageUrls = []
            _imageUrls.append(imageUrl)
            x = downloadImage(_imageUrls[0], x)
            print(x)
            if x == limit:
                break
def makesize(img,targetsize):#most of mine
    print(img.shape)
    print(targetsize)
    if targetsize[0]/img.shape[1] >=targetsize[1]/img.shape[0]:
        othersize=int(img.shape[1]*(targetsize[1]/img.shape[0]))
        
        img=cv.resize(img,(othersize,targetsize[1]))
        print(othersize)
    else:
        othersize=int(img.shape[0]*(targetsize[0]/img.shape[1]))
        
        img=cv.resize(img,(targetsize[0],othersize))
        print(othersize)
    print(img.shape)
    ht, wd, cc= img.shape
    color = (0,0,0)
    result = np.full((targetsize[1],targetsize[0],cc), color, dtype=np.uint8)
    print(result.shape)
    # compute center offset
    xx = (targetsize[0] - wd) // 2
    yy = (targetsize[1] - ht) // 2
    print(xx,yy)
    # copy img image into center of result image
    result[yy:yy+ht, xx:xx+wd] = img
    return result

def getListOfFiles(dirName):#not my code
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

def image_beat_match(beatss):#my code
    global out
    global fps
    global flipflop
    imagelist = getListOfFiles(dir)
    print(imagelist)
    print(type(imagelist))
    for i in range(int(beatss[0])):
        frame=np.zeros((vid_height,vid_width,3), np.uint8)
        #cvSet(frame, CV_RGB(redVal,greenVal,blueVal));
        out.write(frame)
    for beat in beatss[1:]:
        print(beat)
        w=list(beatss).index(beat)
        print("w",w)
        if len(imagelist)<=w:
            print("not enough images. ending")
            break
        j=imagelist[w]
        img=cv.imread(j)
        frame=makesize(img,(vid_width,vid_height))
        print(int((beatss[(w-1)])))
        for i in range(int((beatss[(w-1)])),int(beat)):  
            out.write(frame)
        
        
        
def run ():
    sub=input("What subreddit do you want to use? r/")#ask for sub
    global dir#get global vars
    dir+=sub+"/"#edit global vars to match sub
    print(dir)#debug
    if not os.path.exists(dir):#make dir if not there
        os.makedirs(dir)
    #song input
    #root = Tk()#tk is the file browser for selecting the music
    #root.withdraw()
    #root.filename =  filedialog.askopenfilename(initialdir = ".",title = "Select file",filetypes = (("mp3 files","*.mp3"),("all files","*.*")))
    #song=root.filename #TODO fix this garbage
    #i wish i had more time for this but its 3 am on the 23rd
    #print (root.filename)
    #root.destroy()
    song=input("where is the song you want to use?")
    #y, sr = librosa.load(song)#prev version
    reddit_rip(sub,len(beats(song)))#rip the same number of images as beats
    print("image matching time")#debug
    image_beat_match(beats(song))#pair the images to beats
    out.release#stop editing the video
    #debug the ffmpeg command to sync audio/video
    print("ffmpeg",os.system("cmd.exe \c ffmpeg -y -i output.avi -i "+song+" -map 0:0 -map 1:0 -c:v copy -c:a copy output_with_audio.avi"))
    print("done")#thank god


run()

What subreddit do you want to use? r/aww
reddit_images/aww/
where is the song you want to use?wii.mp3
https://www.reddit.com/r/aww/hot
112
https://v.redd.it/y34xg3ry5f051
0
https://i.redd.it/ljhgis88ue051.jpg
1
https://i.redd.it/utl0pb08we051.jpg
2
https://v.redd.it/qgvsikupve051
2
https://gfycat.com/complexhotdassie.gif
2
https://gfycat.com/heftyfeistyflyingfish
2
https://v.redd.it/5ld9zpyfte051
2
https://i.imgur.com/WNAGgxL.gif
2
https://i.redd.it/iycjrx5kzd051.jpg
3
https://v.redd.it/x90rp30ihd051
3
https://i.imgur.com/gjfCvLV.jpg
4
https://v.redd.it/5y0thijouc051
4
https://v.redd.it/xowxguac6c051
4
https://i.redd.it/mj113q5qcc051.jpg
5
https://v.redd.it/y6qre4s3uf051
5
https://v.redd.it/q2wp02pmyc051
5
https://i.redd.it/doi4x1869e051.jpg
6
https://v.redd.it/siwl2qzfmf051
6
https://i.redd.it/m8p6mpglnd051.jpg
7
https://i.redd.it/kmhtcn7a0cw11.jpg
8
https://i.imgur.com/80W2tcg.jpg
9
https://v.redd.it/kr0ql7i6od051
9
https://v.redd.it/zb9vcm63ed051
9
https://i.redd.it/w01mjpbi6c051.jp

1080
(1080, 1080, 3)
(1080, 1920, 3)
420 0
11
67
w 2
(4032, 3024, 3)
(1920, 1080)
810
(1080, 810, 3)
(1080, 1920, 3)
555 0
36
97
w 3
(4608, 2592, 3)
(1920, 1080)
607
(1080, 607, 3)
(1080, 1920, 3)
656 0
67
130
w 4
(2494, 2000, 3)
(1920, 1080)
866
(1080, 866, 3)
(1080, 1920, 3)
527 0
97
161
w 5
(4032, 3024, 3)
(1920, 1080)
810
(1080, 810, 3)
(1080, 1920, 3)
555 0
130
193
w 6
(4032, 3024, 3)
(1920, 1080)
810
(1080, 810, 3)
(1080, 1920, 3)
555 0
161
224
w 7
(2048, 3938, 3)
(1920, 1080)
998
(998, 1920, 3)
(1080, 1920, 3)
0 41
193
255
w 8
(466, 960, 3)
(1920, 1080)
932
(932, 1920, 3)
(1080, 1920, 3)
0 74
224
287
w 9
(960, 720, 3)
(1920, 1080)
810
(1080, 810, 3)
(1080, 1920, 3)
555 0
255
319
w 10
(4032, 3024, 3)
(1920, 1080)
810
(1080, 810, 3)
(1080, 1920, 3)
555 0
287
350
w 11
(4032, 3024, 3)
(1920, 1080)
810
(1080, 810, 3)
(1080, 1920, 3)
555 0
319
375
w 12
(3616, 3024, 3)
(1920, 1080)
903
(1080, 903, 3)
(1080, 1920, 3)
508 0
350
406
w 13
(444, 700, 3)
(1920, 1080)
1702
(1080, 1702, 3)
(10

3016
w 99
(2767, 2907, 3)
(1920, 1080)
1134
(1080, 1134, 3)
(1080, 1920, 3)
393 0
2983
3044
w 100
(1125, 2436, 3)
(1920, 1080)
886
(886, 1920, 3)
(1080, 1920, 3)
0 97
3016
3078
w 101
(2688, 1242, 3)
(1920, 1080)
499
(1080, 499, 3)
(1080, 1920, 3)
710 0
3044
3110
w 102
(1728, 1296, 3)
(1920, 1080)
810
(1080, 810, 3)
(1080, 1920, 3)
555 0
3078
3141
w 103
(1349, 1080, 3)
(1920, 1080)
864
(1080, 864, 3)
(1080, 1920, 3)
528 0
3110
3171
w 104
(2397, 2397, 3)
(1920, 1080)
1080
(1080, 1080, 3)
(1080, 1920, 3)
420 0
3141
3204
w 105
(716, 720, 3)
(1920, 1080)
1086
(1080, 1086, 3)
(1080, 1920, 3)
417 0
3171
3236
w 106
(2745, 1836, 3)
(1920, 1080)
722
(1080, 722, 3)
(1080, 1920, 3)
599 0
3204
3265
w 107
(3128, 1852, 3)
(1920, 1080)
639
(1080, 639, 3)
(1080, 1920, 3)
640 0
3236
3292
w 108
(595, 750, 3)
(1920, 1080)
1361
(1080, 1361, 3)
(1080, 1920, 3)
279 0
3265
3322
w 109
(3024, 4032, 3)
(1920, 1080)
1440
(1080, 1440, 3)
(1080, 1920, 3)
240 0
3292
3354
w 110
(3024, 4032, 3)
(1920, 1080)
1440
(1080

In [ ]:
import cv2 as cv
import os
import numpy as np
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles
img=cv.imread(getListOfFiles(dir)[0])
cv.imshow("yeet",img)

In [5]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [10]:
print(beatss[0]*fps)

NameError: name 'beatss' is not defined

In [12]:
os.system("cmd.exe \c ffmpeg -y -i output.avi -i "+"song.mp3"+" -map 0:0 -map 1:0 -c:v copy -c:a copy output_with_audio.avi")


0